In [66]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv("data/troff-v1.0.tsv",  sep='\t')

In [7]:
data.head()

,id,timestamp,text,label
0,973568937593065472,1520952977415,@USER06095 Hırsız demişken Tuncay sizin şu 1.2...,grp
1,973568937723035648,1520952977446,Ne bileyim sen hastayım deyince bende veterine...,ind
2,973568937911873536,1520952977491,Akşam eve gittiğimizde yorgunluğuma iyi gelece...,grp
3,973568939925090304,1520952977971,Kook’un sesini 18381 kez dinledikten sonra eğe...,prof
4,973568940667539457,1520952978148,@USER05270 @USER04816 o macta adam 6 7 tane ne...,grp


In [22]:
# make all text lowercase
# tum yazilari kucuk harfe cevir
data['text'] = data['text'].str.lower()

# remove usernames
# kullanici adlarini sil
data['text'] = data['text'].map(lambda x: re.sub('@[^\s]+','',x))


0         hırsız demişken tuncay sizin şu 1.200 bin tl'...
1        ne bileyim sen hastayım deyince bende veterine...
2        akşam eve gittiğimizde yorgunluğuma iyi gelece...
3        kook’un sesini 18381 kez dinledikten sonra eğe...
4          o macta adam 6 7 tane net sut cikartti aksin...
                               ...                        
35279                    great priest elmalara pot basıyor
35280    alkollüyken yazmaya harika bir çözüm sunuyorum...
35281     cvp vermek istemedim ama dayanamıyor insan ta...
35282    bazı takip ettiğim kişileri aha bu da hasta am...
35283       allah akıl fakirliği vermesin geri hallolur :d
Name: text, Length: 35284, dtype: object

In [25]:
# create labels vector for training
# modeli egitmek icin siniflandirmalari olustur
data['label_sinkaf'] = data['label'] != 'non'

In [34]:
# 6.8K offensive - kufurlu yorum
# 28K  non-offensive - kufursuz yorum
data['label_sinkaf'].value_counts()

False    28439
True      6845
Name: label_sinkaf, dtype: int64

In [72]:
stop_words_tr = pd.read_csv("https://raw.githubusercontent.com/ahmetax/trstop/master/dosyalar/turkce-stop-words", header=None)

stop_words_tr = stop_words_tr[0].to_numpy()

In [169]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [170]:
vectorizer = CountVectorizer(stop_words=frozenset(stop_words_tr), min_df=0.001,  ngram_range= (1, 1))

X = vectorizer.fit_transform(data['text'])
y = data['label_sinkaf']

X_train, X_test, y_train, y_test = train_test_split(X,y)


In [171]:
model = LogisticRegression(class_weight="balanced", dual=False, tol=1e-2, max_iter=1e5)

# Train the model
model = LinearSVC(class_weight="balanced", dual=False, tol=1e-2, max_iter=1e5)
cclf = CalibratedClassifierCV(base_estimator=model)
cclf.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=LinearSVC(class_weight='balanced',
                                                dual=False, max_iter=100000.0,
                                                tol=0.01))

In [187]:
print("Train acc:\t{0:.3f}".format(cclf.score(X_train, y_train)))
print("Test acc:\t{0:.3f}".format(cclf.score(X_test, y_test)))

Train acc:	0.842
Test acc:	0.841


In [188]:
# Offensive? - Kufur mu?
cclf.predict(vectorizer.transform([
    "abi cok iyi", 
    "şerefsiz", 
    "göt herif yahu", 
    "sen ne güzel bir detaysın!"]))

array([False,  True,  True, False])

In [189]:
cclf.predict_proba(vectorizer.transform([
    "abi cok iyi", 
    "şerefsiz", 
    "göt herif yahu", 
    "sen ne güzel bir detaysın!"]))

array([[0.83311491, 0.16688509],
       [0.49288601, 0.50711399],
       [0.07407014, 0.92592986],
       [0.86621124, 0.13378876]])